In [1]:
# Import libraries. 
# Generate images in the notebook
%matplotlib inline

import matplotlib.pyplot as plt
import collections
from collections import defaultdict
import gzip
import itertools
import numpy as np
import os
import time
import seaborn as sns
import pandas as pd

from ipywidgets import interact
import bokeh
import bokeh.io
from bokeh.io import push_notebook
from bokeh.plotting import figure, show, save, output_notebook, output_file

# Import colour palettes for later on
from bokeh.palettes import Category20b
from bokeh.palettes import Purples
from bokeh.palettes import Greens
from bokeh.palettes import YlOrBr
from bokeh.palettes import YlOrRd
from bokeh.palettes import PuOr
from bokeh.palettes import RdGy

# Dimension reduction tools
from sklearn.decomposition import PCA as PCA
from sklearn.manifold import TSNE
import umap 

In [2]:
# Specify your parent directory. This is where your 1KGP files are stored
data_dir = '/Users/kuzel/Beyond/DimRedTut/'

# These are the names of the files we use
vcf_name = 'ALL.wgs.nhgri_coriell_affy_6.20140825.genotypes_has_ped.vcf.gz'
pop_desc_name = '20131219.populations.tsv'
pop_file_name = 'affy_samples.20141118.panel'

vcf_file = os.path.join(data_dir, vcf_name)
population_description_file = os.path.join(data_dir, pop_desc_name)
population_file = os.path.join(data_dir, pop_file_name)

In [3]:
# Code to read in the SNP data. Assign every SNP a value of {0,1,2} relative to reference genome.
from collections import Counter

class snp(object):

    def __init__(self, line, select=False, autosome_only =True):
        """The initialization method takes in a line from the vcf file, as a string, 
        and records the relevant information. 
        line: a string from a vcf file
        select: a list of positions of individuals to be analyzed, where positions run from 0 to 
        nInd-1, the number of individuals
        """ 
        
        split_line = line.split()  #  First break down the line into a list of each field
        
        self.failed = False  # A label that we will set to True if something goes wrong.
        
        if line.startswith('#'):
            self.failed = True
            self.failure_cause = "line was a header line, not a snp"
            return
        
        if len(split_line)<=5:
            self.failed = True
            self.failure_cause = "incorrectly formatted line, should have at least 5 fields " + line
            return
          
        self.chrom = split_line[0]
        if autosome_only:
            if self.chrom not in ["%d" % (i,) for i in range(1,23)]:
                self.failed = True
                self.failure_cause = "not recognized as an autosome while autosome_only set to True"
                return
        
        self.chrom = int(split_line[0]) # Chromosome (numbered)
        self.position = int(split_line[1])  # The coordinates of the snp
        self.rid = split_line[2] # Name/Record ID
        self.ref_allele = split_line[3]
        self.alt_allele = split_line[4] # The alterate allele according to the vcf; also a string 
        # Only accept snps in ACGT. 
        if self.ref_allele not in ["A","C","G","T"] or self.alt_allele not in ["A","C","G","T"]:
            self.failed = True
            self.failure_cause = "ref or alt not in ACGT"
            return
        self.filter = split_line[6]  # See vcf format specifications for the interpretation of 
                                    # the filter field
        if self.filter not in ['PASS', '.'] :  # PASS indicates a SNP that passed all QC filters.
            self.failed = True
            self.failure_cause = self.filter
            return
              
        self.genotype_strings = split_line[9:]

        # Prepare a list that will contain the transformed genotypes. 
        # Since we already know how long the list will be, it makes sense 
        # to create an array of zeros of the same length as self.gtypes, 
        
        self.genotype_array = np.zeros(len(self.genotype_strings), dtype = np.int8)             

        # Count the number of each genotype. 
        # There may be different strings giving the same genotype so we increment the 
        # counts found so far for the genotype by the number of times the  
        # For example, "0/0" and "0\0" give homref, and "0|1" and "1|0" give het
        
        n_missing = 0
        for index,genotype_string in enumerate(self.genotype_strings):
            if genotype_string == './.':
                n_missing +=1 
                self.genotype_array[index]=-1
                continue # missing data will be left as 0
            allele_0 = genotype_string[0] # Get the first allele (as a string)
            allele_1 = genotype_string[2]
            if (allele_0=='1' and allele_1=='1'): # Use rstrip because windows machines will occasionally have extra \n
                self.genotype_array[index]=2
            elif ((allele_0=='0' and allele_1=='1') or (allele_0=='1' and allele_1=='0')):
                self.genotype_array[index]=1   
            elif (allele_0=='0' and allele_1=='0'):
                # The array was initialized to zero, so nothing to do here!
                continue
            else:
                print(("unknown genotype", genotype_string))
                self.failed=True
                self.failedreason="unknown genotype"
                return

In [4]:
#The following step imports the genotype data. It is not particularly efficient so will take a few minutes even if we skip some of the lines.
# Specify the number of lines to skip to avoid storing every line in memory
number_of_lines_to_skip = 5

start_time = time.time()

genotype_matrix = []  # Will contain our numerical genotype matrix. 
genotype_positions = []
genotype_names = []
x = 0
error_count = 0

with gzip.open(vcf_file,'rt') as f:
    count = 0
    for line in f:
        count+=1
        if count % number_of_lines_to_skip == 0:
            if line.startswith("#") or snp(line).failed:
                if snp(line).failure_cause != "line was a header line, not a snp":
                    error_count += 1
                    if x < 10:
                        print('Failed: ' + snp(line).failure_cause)
                        x+=1
                continue
            
            return_snp = snp(line)
            genotype_matrix.append(return_snp.genotype_array)
            genotype_names.append(return_snp.rid)
            genotype_positions.append([return_snp.chrom, return_snp.position])

end_time = time.time()
            
print("Run time in seconds: " + str(end_time - start_time))

Failed: not recognized as an autosome while autosome_only set to True
Failed: not recognized as an autosome while autosome_only set to True
Failed: not recognized as an autosome while autosome_only set to True
Failed: not recognized as an autosome while autosome_only set to True
Failed: not recognized as an autosome while autosome_only set to True
Failed: not recognized as an autosome while autosome_only set to True
Failed: not recognized as an autosome while autosome_only set to True
Failed: not recognized as an autosome while autosome_only set to True
Failed: not recognized as an autosome while autosome_only set to True
Failed: not recognized as an autosome while autosome_only set to True
Run time in seconds: 173.00819206237793


In [5]:
print(len(genotype_matrix))
print(genotype_matrix[1])
print(genotype_matrix[1].shape)

173727
[0 0 0 ... 0 0 0]
(3450,)


In [6]:
# Transpose the matrix
transposed_genotype_matrix = np.array(genotype_matrix).transpose()

In [7]:
#The following code imports auxiliary data (populations, continent, descriptive data, colouring, etc.)
population_by_individual = defaultdict(int)
individuals_by_population = defaultdict(list)  # A dictionary containing all the individuals in a given population

for line in open(population_file,'r'):
    split_line = line.split()
    if split_line[0] == 'sample':  # header line
        continue

    sample_name = split_line[0]
    population_name = split_line[1]
    population_by_individual[sample_name] = population_name
    individuals_by_population[population_name].append(sample_name) 

populations = list(individuals_by_population.keys())

In [8]:
# The path to the place where you put the population name file.
name_by_code = {}  # A dictionary giving the full name of each population code
pop_by_continent = {}  # A dictionary giving the code of each population within a continent  
continent_by_population = {}  # A dictionary giving the continent for each population code
for line in open(population_description_file,'r'):
    split_line = line.split('\t')
    if split_line[0] in ['Population Description','Total','']:  # header or footer
        continue
    name_by_code[split_line[1]] = split_line[0]
    continent_by_population[split_line[1]] = split_line[2]
    try: 
        pop_by_continent[split_line[2]].append(split_line[1])
    except KeyError:
        pop_by_continent[split_line[2]] = [split_line[1]]

continents = list(pop_by_continent.keys()) 
    
    
# Populations listed by continent
pops=[]
for continent in continents:
    pops.extend(pop_by_continent[continent])

In [9]:
# Assign colours to each population, roughly themed according to continent
# The Category20b palette has a bunch of groups of 4 shades in the same colour range
color_dict = {}
for i, cont in enumerate(continents): 
    for j, pop in enumerate(pop_by_continent[cont]):
        color_dict[pop] = Category20b[20][4*i+j%4]

# Colour palette above only really supports groups of 4 so we have to manually specify a few colours for the 5th/6th
# members of a group

color_dict['CHS'] = Purples[9][4]# purple
color_dict['STU'] = Greens[9][6] # green
color_dict['LWK'] = PuOr[11][-1] # brown
color_dict['MSL'] = PuOr[11][-2] # rusty brown
color_dict['YRI'] = PuOr[11][-3] # cappucino w/ extra milk (stirred)
color_dict['CEU'] = RdGy[11][-3]

In [10]:
for line in gzip.open(vcf_file,'rt'):
    if line.startswith("#"):
        if not line.startswith("##"):
            # Extract the individuals for the population, as a list of strings
            # Windows users may have trailing \n characters
            individuals = line.split()[9:]
            # Once we've extracted the individuals, we can exit the loops. 
            break

# Build a list of populations for each indiviudal in the vcf file
lspop = []
for ind in individuals:
    pop = population_by_individual[ind]
    if pop == 0:
        lspop.append("missing")
    else:
        lspop.append(pop)

        
indices_of_population_members = defaultdict(list)

for index,individual in enumerate(individuals):
    try:
        indices_of_population_members[population_by_individual[individual]].append(index)
    except KeyError: # We do not have population info for this individual
        continue

In [11]:
count = 0

for p in pop_by_continent:
    count+=len(pop_by_continent[p])
    
print(count)

26


In [12]:
imax = 0
imin = 200
for i in indices_of_population_members:
    imax = max(len(indices_of_population_members[i]),imax)
    imin = min(len(indices_of_population_members[i]),imin)
    
print(imax, imin)

183 104


In [13]:
#Dimension reduction
#First we generate our prinicipal component projection

# Calculate the PC axes
# This will take a few minutes
pca_full = PCA().fit(transposed_genotype_matrix)

In [14]:
# Project onto the PC axes
proj_pca = pca_full.fit_transform(transposed_genotype_matrix)

In [ ]:
#The following code uses non-PCA methods. We explore t-SNE and UMAP specifically. For t-SNE you can work in 2 or 3 dimensions. You can also specify a variety of hyperparameters (e.g. perplexity to control "tightness" of clusters) but the defaults are sufficient for our purposes.
# Project the genotype data matrix to two dimensions via t-SNE. This may take several minutes to run.
proj_tsne_gt = TSNE(n_components = 2).fit_transform(transposed_genotype_matrix)

In [ ]:
# Project the genotype data matrix to two dimensions via UMAP
proj_umap_gt = umap.UMAP(n_components=2, n_neighbors=15, min_dist=0.5).fit_transform(transposed_genotype_matrix)

In [15]:
# Number of principal components to use
n_pc = 15

In [16]:
# Project the principal components via t-SNE to 2 dimensions.
proj_tsne_pca = TSNE(n_components=2).fit_transform(proj_pca[:,:n_pc])

/Users/kuzel/anaconda3/envs/beyondpca/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/Users/kuzel/anaconda3/envs/beyondpca/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


In [17]:
# Project the principal components via UMAP to 2 dimensions.
proj_umap_pca = umap.UMAP(n_components=2, n_neighbors=15, min_dist=0.5).fit_transform(proj_pca[:,:n_pc])

/Users/kuzel/anaconda3/envs/beyondpca/lib/python3.10/site-packages/sklearn/manifold/_spectral_embedding.py:260: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


In [ ]:
n_pc = 25
proj_umap_pca_2 = umap.UMAP(n_components=2, n_neighbors=15, min_dist=0.5).fit_transform(proj_pca[:,:n_pc])

In [18]:
#Generate interactive HTML files

# Use the PCA projection
dset = proj_pca

# Select the dimensions in the projection
dim1 = 0
dim2 = 1

p = figure(plot_width=1350, plot_height=800)
p.title.text = 'PCA projection: PC' + str(dim1+1) + ' vs PC' + str(dim2+1)

for cont in continents: 
    for pop in pop_by_continent[cont]:
        projections_within_population = dset[indices_of_population_members[pop]]
        p.circle(projections_within_population[:,dim1], projections_within_population[:,dim2], 
                 legend=name_by_code[pop], color = color_dict[pop])

p.legend.location = "top_left"
p.legend.click_policy="hide"

output_file("interactive_pca.html", title="PCA projection")

save(p)
show(p)


In [ ]:
# Use the UMAP projection of the genotype data
dset = proj_umap_gt

dim1 = 0
dim2 = 1

p = figure(plot_width=1350, plot_height=800)
p.title.text = 'UMAP projection of genotype data'

for cont in continents: 
    for pop in pop_by_continent[cont]:
        projections_within_population = dset[indices_of_population_members[pop]]
        p.circle(projections_within_population[:,dim1], projections_within_population[:,dim2], 
                 legend=name_by_code[pop], color = color_dict[pop])

p.legend.location = "top_left"
p.legend.click_policy="hide"

output_file("interactive_umap.html", title="UMAP projection")

save(p)
show(p)

In [ ]:
# Use the tSNE projection of the genotype data
dset = proj_tsne_gt

dim1 = 0
dim2 = 1

p = figure(plot_width=1350, plot_height=800)
p.title.text = 't-SNE projection of genotype data'

for cont in continents: 
    for pop in pop_by_continent[cont]:
        projections_within_population = dset[indices_of_population_members[pop]]
        p.circle(projections_within_population[:,dim1], projections_within_population[:,dim2], 
                 legend=name_by_code[pop], color = color_dict[pop])

p.legend.location = "top_left"
p.legend.click_policy="hide"

output_file("interactive_tsne.html", title="t-SNE projection")

save(p)
show(p)

In [19]:
# Use the UMAP projection of the PCs
dset = proj_umap_pca

dim1 = 0
dim2 = 1

p = figure(plot_width=1350, plot_height=800)
p.title.text = 'UMAP projection of principal components'

for cont in continents: 
    for pop in pop_by_continent[cont]:
        projections_within_population = dset[indices_of_population_members[pop]]
        p.circle(projections_within_population[:,dim1], projections_within_population[:,dim2], 
                 legend=name_by_code[pop], color = color_dict[pop])

p.legend.location = "top_left"
p.legend.click_policy="hide"

output_file("interactive_pca_umap.html", title="PCA-UMAP projection")

save(p)
show(p)

In [20]:
# Use the tSNE projection of the PCs
dset = proj_tsne_pca

dim1 = 0
dim2 = 1

p = figure(plot_width=1350, plot_height=800)
p.title.text = 't-SNE projection of principal components'

for cont in continents: 
    for pop in pop_by_continent[cont]:
        projections_within_population = dset[indices_of_population_members[pop]]
        p.circle(projections_within_population[:,dim1], projections_within_population[:,dim2], 
                 legend=name_by_code[pop], color = color_dict[pop])

p.legend.location = "top_left"
p.legend.click_policy="hide"

output_file("interactive_pca_tsne.html", title="PCA-TSNE projection")

#save(p)
show(p)